# Calcul du coût en fonction du tarif choisi

A partir des différents tarifs que j'ai receuilli, et de la consommation effective, je calcul le coût pour chaque tarif. 

In [1]:
import pandas as pd

df = pd.read_parquet("datalake/enedis-data-ready-to-work.parquet")


In [2]:
repartition_consommation = pd.DataFrame(
    {
        "Consommation par periode en kWh": [
            df["Différence journalier Heure Pleine"].sum() / 1000,
            df["Différence journalier Heure Creuse"].sum() / 1000,
            df["Différence journalier Weekend"].sum() / 1000,
        ],
        "Périodes": ["Heure Pleine", "Heure Creuse", "Weekend"],
    }
)
repartition_consommation


,Consommation par periode en kWh,Périodes
0,132.935,Heure Pleine
1,158.837,Heure Creuse
2,136.113,Weekend


In [ ]:
import plotly.express as px

fig = px.pie(
    repartition_consommation,
    values="Consommation par periode en kWh",
    names="Périodes",
    title="Consommation par période",
)
fig.show()


![Répartition de la consommation](images/repartition-consommation.png)

In [4]:
df_tarif_vert_base = pd.read_excel("datalake/tarif-vert-base.ods")
selected_df_tarif_vert_base = df_tarif_vert_base.loc[df_tarif_vert_base["kva"] == 9]
selected_df_tarif_vert_base.reset_index(drop=True, inplace=True)
cout_tarif_vert_base = (df["Différence journalier Total Consommation"] / 1000) * (
    selected_df_tarif_vert_base["prix kwh"].values[0] / 100
)
calculated_cout_tarif_vert_base = cout_tarif_vert_base.sum() + (
    (selected_df_tarif_vert_base["abonnement"].values[0])
)


In [5]:
df_tarif_bleu_base = pd.read_excel("datalake/tarif-bleu-base.ods")
selected_df_tarif_bleu_base = df_tarif_bleu_base.loc[df_tarif_bleu_base["kva"] == 9]
selected_df_tarif_bleu_base.reset_index(drop=True, inplace=True)
cout_tarif_bleu_base = (df["Différence journalier Total Consommation"] / 1000) * (
    selected_df_tarif_bleu_base["prix kwh"].values[0] / 100
)
calculated_cout_tarif_bleu_base = cout_tarif_bleu_base.sum() + (
    (selected_df_tarif_bleu_base["abonnement"].values[0])
)


In [6]:
df_tarif_vert_weekend = pd.read_excel("datalake/tarif-vert-weekend.ods")
selected_df_tarif_vert_weekend = df_tarif_vert_weekend.loc[
    df_tarif_vert_weekend["kva"] == 9
]
selected_df_tarif_vert_weekend.reset_index(drop=True, inplace=True)
cout_tarif_vert_weekend = pd.DataFrame(columns=["cout heure semaine", "cout weekend"])
cout_tarif_vert_weekend["cout heure semaine"] = (
    df["Différence journalier Heure Creuse"] / 1000
) * (selected_df_tarif_vert_weekend["heure semaine"].values[0] / 100) + (
    df["Différence journalier Heure Pleine"] / 1000
) * (
    selected_df_tarif_vert_weekend["heure semaine"].values[0] / 100
)
cout_tarif_vert_weekend["cout weekend"] = (
    df["Différence journalier Weekend"] / 1000
) * (selected_df_tarif_vert_weekend["heure weekend"].values[0] / 100)
cout_tarif_vert_weekend["total"] = (
    cout_tarif_vert_weekend["cout weekend"]
    + cout_tarif_vert_weekend["cout heure semaine"]
)
calculated_cout_tarif_vert_weekend = cout_tarif_vert_weekend["total"].sum() + (
    (selected_df_tarif_vert_weekend["abonnement"].values[0])
)


In [7]:
df_tarif_vert_weekend_hp_hc = pd.read_excel("datalake/tarif-vert-weekend-hc-hp.ods")
selected_df_tarif_vert_weekend_hp_hc = df_tarif_vert_weekend_hp_hc.loc[
    df_tarif_vert_weekend_hp_hc["kva"] == 9
]
selected_df_tarif_vert_weekend_hp_hc.reset_index(drop=True, inplace=True)

cout_tarif_vert_weekend_hp_hc = pd.DataFrame(
    columns=["cout heure creuse", "cout heure pleine", "cout weekend"]
)
cout_tarif_vert_weekend_hp_hc["cout heure creuse"] = (
    df["Différence journalier Heure Creuse"] / 1000
) * (selected_df_tarif_vert_weekend_hp_hc["heure creuse"].values[0] / 100)
cout_tarif_vert_weekend_hp_hc["cout heure pleine"] = (
    df["Différence journalier Heure Pleine"] / 1000
) * (selected_df_tarif_vert_weekend_hp_hc["heure pleine"].values[0] / 100)
cout_tarif_vert_weekend_hp_hc["cout weekend"] = (
    df["Différence journalier Weekend"] / 1000
) * (selected_df_tarif_vert_weekend_hp_hc["heure weekend"].values[0] / 100)
cout_tarif_vert_weekend_hp_hc["total"] = (
    cout_tarif_vert_weekend_hp_hc["cout weekend"]
    + cout_tarif_vert_weekend_hp_hc["cout heure pleine"]
    + cout_tarif_vert_weekend_hp_hc["cout heure creuse"]
)
calculated_cout_tarif_vert_weekend_hp_hc = cout_tarif_vert_weekend_hp_hc[
    "total"
].sum() + ((selected_df_tarif_vert_weekend_hp_hc["abonnement"].values[0]))


In [ ]:
comparatif = pd.DataFrame(
    {
        "Coût": pd.Series(
            [
                calculated_cout_tarif_bleu_base,
                calculated_cout_tarif_vert_base,
                calculated_cout_tarif_vert_weekend,
                calculated_cout_tarif_vert_weekend_hp_hc,
            ]
        ).round(2),
        "Tarif": [
            "Tarif Bleu Base",
            "Tarif Vert Base",
            "Tarif Vert Weekend",
            "Tarif Vert Weekend Heure Creuse",
        ],
    }
)
import plotly.express as px

fig = px.bar(
    comparatif,
    x="Tarif",
    y="Coût",
    text="Coût",
    range_y=[85, 97],
    title="Coût en euros en fonction du tarif choisi",
)
fig.show()


![Fallback pour Github, calcul cout électricité en fonction du tarif](images/calcul-cout-electricite.png)